In [1]:
import pandas as pd
import json
from pprint import pprint
import numpy as np
from sqlalchemy import create_engine

In [ ]:
#Extract section

In [3]:
#save .csv file as pandas df
happiness2017 = pd.read_csv("2017.csv")
happiness2017.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [ ]:
#save .json file object
with open('factbook-country-profiles.json', encoding="utf-8-sig") as f:
    factbook = json.load(f)

#pprint(factbook)

In [ ]:
#Examples of locating values in JSON file

In [44]:
#population value
factbook['countries']['afghanistan']['data']['people']['population']['total']

34124811

In [45]:
#population rank
factbook['countries']['afghanistan']['data']['people']['population']['global_rank']

39

In [43]:
#inflation rate (in %)
factbook['countries']['afghanistan']['data']['economy']['inflation_rate']['annual_values'][0]['value']

5

In [47]:
#unemployment rate (in %)
factbook['countries']['afghanistan']['data']['economy']['unemployment_rate']['annual_values'][0]['value']

23.9

In [53]:
#population without electricity
factbook['countries']['afghanistan']['data']['energy']['electricity']['access']['population_without_electricity']['value']

18999254

In [57]:
#type of government
factbook['countries']['afghanistan']['data']['government']['government_type']

'presidential Islamic republic'

In [62]:
#percent of population with access to internet
factbook['countries']['afghanistan']['data']['communications']['internet']['users']['percent_of_population']

10.6

In [ ]:
#Transform section

In [161]:
#add columns to dataframe
happiness2017["Population"] = ""
happiness2017["Population Rank"] = ""
happiness2017["Inflation Rate"] = ""
happiness2017["Unemployment Rate"] = ""
happiness2017["Population without Electricity"] = ""
happiness2017["Type of Government"] = ""
happiness2017["Access to Internet"] = ""

In [162]:
country_name_dict = {'czech_republic':'czechia',
                     'taiwan_province_of_china':'taiwan',
                     'south_korea':'korea_south',
                     'hong_kong_s.a.r.,_china':'hong_kong',
                     'myanmar':'burma',
                     'congo_(brazzaville)':'congo_republic_of_the',
                     'congo_(kinshasa)':'congo_democratic_republic_of_the',
                     'ivory_coast':"cote_d'_ivoire"}

for index, row in happiness2017.iterrows():
    country = row['Country'].lower().replace(' ','_')
    try:
        if not country in factbook['countries']:
            country = country_name_dict[country]

        pop = int(factbook['countries'][country]['data']['people']['population']['total'])
    except:
        pop = np.nan
        
    try:
        pop_rank = int(factbook['countries'][country]['data']['people']['population']['global_rank'])
    except:
        pop_rank = np.nan
        
    try:
        inflation = float(factbook['countries'][country]['data']['economy']['inflation_rate']['annual_values'][0]['value'])
    except:
        inflation = np.nan
        
    try:    
        unemployment = float(factbook['countries'][country]['data']['economy']['unemployment_rate']['annual_values'][0]['value'])
    except:
        unemployment = np.nan
        
    try:
        pop_electric = float(factbook['countries'][country]['data']['energy']['electricity']['access']['population_without_electricity']['value'])
    except:
        pop_electric = np.nan

    try:
        gov_type = factbook['countries'][country]['data']['government']['government_type']
    except:
        gov_type = np.nan
        
    try:
        internet = float(factbook['countries'][country]['data']['communications']['internet']['users']['percent_of_population'])
    except:
        internet = np.nan
        
    happiness2017.loc[index,'Population'] = pop
    happiness2017.loc[index,'Population Rank'] = pop_rank
    happiness2017.loc[index,'Inflation Rate'] = inflation
    happiness2017.loc[index,'Unemployment Rate'] = unemployment
    happiness2017.loc[index,'Population without Electricity'] = pop_electric
    happiness2017.loc[index,'Type of Government'] = gov_type
    happiness2017.loc[index,'Access to Internet'] = internet

In [166]:
happiness2017 = happiness2017.drop(axis=1,columns=['Happiness.Score','Whisker.high','Whisker.low','Trust..Government.Corruption.','Dystopia.Residual'])
happiness2017 = happiness2017.rename(index=str,columns={'Country':'country', 
                      'Happiness.Rank':'happiness_rank',
                      'Economy..GDP.per.Capita.':'economy', 
                      'Family':'family',
                      'Health..Life.Expectancy.':'health', 
                      'Freedom':'freedom', 
                      'Generosity':'generosity', 
                      'Population':'population_total',
                      'Population Rank':'population_global_rank', 
                      'Inflation Rate':'inflation_rate', 
                      'Unemployment Rate':'unemployment_rate',
                      'Population without Electricity':'population_without_electricity', 
                      'Type of Government':'type_of_gov',
                      'Access to Internet':'percent_population_internet'})

In [ ]:
#Load section

In [163]:
#connect to sql database
connection_string = 'root:Baseball.1@127.0.0.1/project_etl'
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [172]:
#load pandas df to sql
happiness2017.to_sql(name='happiness', con=engine, if_exists='replace', index=False)

In [ ]:
#Example queries

In [173]:
#select all from sql
pd.read_sql_query('select * from happiness', con=engine).head()

,country,happiness_rank,economy,family,health,freedom,generosity,population_total,population_global_rank,inflation_rate,unemployment_rate,population_without_electricity,type_of_gov,percent_population_internet
0,Norway,1,1.616463,1.533524,0.796667,0.635423,0.362012,5320045.0,120.0,1.9,4.0,NaN,parliamentary constitutional monarchy,97.3
1,Denmark,2,1.482383,1.551122,0.792566,0.626007,0.355280,5605948.0,116.0,1.1,5.8,NaN,parliamentary constitutional monarchy,97.0
2,Iceland,3,1.480633,1.610574,0.833552,0.627163,0.475540,339747.0,178.0,1.8,2.8,NaN,parliamentary republic,98.2
3,Switzerland,4,1.564980,1.516912,0.858131,0.620071,0.290549,8236303.0,98.0,0.5,3.0,NaN,federal republic (formally a confederation),89.4
4,Finland,5,1.443572,1.540247,0.809158,0.617951,0.245483,5518371.0,117.0,0.8,8.6,NaN,parliamentary republic,87.7


In [177]:
#top 10 happiest countries
pd.read_sql_query('select country, happiness_rank, population_total, type_of_gov from happiness limit 10', con=engine)

,country,happiness_rank,population_total,type_of_gov
0,Norway,1,5320045,parliamentary constitutional monarchy
1,Denmark,2,5605948,parliamentary constitutional monarchy
2,Iceland,3,339747,parliamentary republic
3,Switzerland,4,8236303,federal republic (formally a confederation)
4,Finland,5,5518371,parliamentary republic
5,Netherlands,6,17084719,parliamentary constitutional monarchy; part of...
6,Canada,7,35623680,federal parliamentary democracy (Parliament of...
7,New Zealand,8,4510327,parliamentary democracy (New Zealand Parliamen...
8,Sweden,9,9960487,parliamentary constitutional monarchy
9,Australia,10,23232413,parliamentary democracy (Federal Parliament) u...


In [181]:
#avg pop_rank by type of government
pd.read_sql_query('select type_of_gov, avg(population_global_rank) from happiness group by type_of_gov', con=engine)

,type_of_gov,avg(population_global_rank)
0,None,NaN
1,absolute monarchy,95.0000
2,communist party-led state,1.0000
3,communist state,15.0000
4,constitutional federal republic,3.0000
5,constitutional monarchy,163.6667
6,constitutional monarchy (emirate),139.0000
7,constitutional monarchy; note - interim milita...,20.0000
8,federal parliamentary constitutional monarchy\...,41.0000
9,federal parliamentary democracy (Parliament of...,38.0000
